In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime


from datetime import datetime, date
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from math import sqrt
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
FILES_DIR = './files/'
events = pd.read_csv(FILES_DIR + 'sampled_events.csv')
product = pd.read_csv(FILES_DIR + 'sampled_products.csv')
users = pd.read_csv(FILES_DIR + 'sampled_users.csv')

In [ ]:
events_df = events.copy()
events_sample = events_df.sample(frac=0.005, random_state=42)
events_sample

In [4]:
# 의미없는 컬럼 삭제 및 null값 처리
drop_events = events_sample.copy()
drop_events['mobile_brand_name'].fillna('un_brand_name', inplace=True)
drop_events.drop('mobile_marketing_name', axis=1, inplace=True)
drop_events.drop('mobile_model_name', axis=1, inplace=True)
drop_events.drop('operating_system_version', axis=1, inplace=True)
drop_events['rating'] = drop_events['event_name']
re_name = {'rating' : {'click_item':1, 'like_item':2, 'add_to_cart':3, 'purchase_success':4}}
drop_events = drop_events.replace(re_name)
drop_events.dropna(axis=0, inplace=True)
df_events = drop_events.copy()
df_events

,session_id,event_timestamp,event_name,user_no,item_no,device_type,mobile_brand_name,country,region,platform,rating
1916969,434cf67167c5becc6321d03bdecd6c60,2021-06-24 09:25:17.097027,click_item,qwTrxFylir2ynPu9N7/g/w==,/ZasVJSuvWNKGQXzKT0uCg==,mobile,Apple,South Korea,Seoul,IOS,1
4178142,23288f3e8d799799645beb4bff6c35ad,2021-07-16 22:38:19.169111,click_item,OXbjPQaxKL7O8HoSMzJtiQ==,/YlZZk+EM3eAaLCGdgyiag==,mobile,Apple,South Korea,Seoul,IOS,1
644115,dc7c3da68a67b1f43a2226ea2733bebf,2021-06-09 23:25:54.245213,click_item,1jhscZ7MrX7w6sojiWm8cw==,TByIyj8aQeeoE2OnGWr4NQ==,mobile,Samsung,South Korea,Seoul,ANDROID,1
2047331,19422b436106a1cbc45bbe12bc59d11a,2021-06-25 22:37:42.032078,click_item,6+Uf31nW96b3VC61EfO4SQ==,F+25W5Hy9tVYnBlffA2Cmw==,mobile,Apple,South Korea,Chungcheongnam-do,IOS,1
2679735,1a963aba0fb154be005ab97ddbf780aa,2021-07-02 08:51:53.27311,click_item,Zfy9Vola1OIOtuunlXev+A==,VoEeDect6p+wMXMoE5zAkw==,mobile,Samsung,South Korea,Seoul,ANDROID,1
...,...,...,...,...,...,...,...,...,...,...,...
594921,4b2b5f6db5589fa7f736b0bd31f87685,2021-06-09 14:33:30.903094,click_item,3df2n7oVw8oZ3MuLxK5KYg==,GD6DtsasqVJamffP55apQA==,mobile,Apple,South Korea,Gyeonggi-do,IOS,1
1256645,288d50850a8b0e43f2f2e6226a33e53a,2021-06-17 08:53:13.703111,click_item,2d9zQSt+DExdfvP/mj3Juw==,OCVwIGJhWNYNIwAxdI/Lag==,mobile,Samsung,South Korea,Seoul,ANDROID,1
4173205,95cdce3d28de536e37dc837cedb58f21,2021-07-16 21:55:49.05804,add_to_cart,fxAptePuBEdVPekHLnmFnw==,mAznRa3p9hv4rhWkunWMFQ==,mobile,Samsung,South Korea,Gyeonggi-do,ANDROID,3
3952661,5efe98cef53495a0ccec290e57e367fa,2021-07-14 17:17:28.635075,click_item,0h0lKPYrJcPeQlwIjKRohg==,tKZFh9pXCjI7VKLT3Pt1tw==,mobile,Apple,South Korea,Jeollabuk-do,IOS,1


In [5]:

today = pd.Timestamp.now().floor('D')
users['birth_date'] = pd.to_datetime(users['birth_date'], errors='coerce')
users = users[~pd.isnull(users['birth_date'])] # remove rows with missing birth date
users['age'] = ((today - users['birth_date']).dt.days / 365.25).astype(int)

def age_category(age):
    age = (age // 10) * 10
    return age

users['age'] = users.age.apply(age_category)

In [6]:
total_df = pd.merge(df_events, users)
total_df = pd.merge(total_df, product)
total_df['birth_date'].fillna('un_birth_date', inplace=True)
total_df['gender'].fillna('un_gender', inplace=True)
total_df.isnull().sum()

session_id           0
event_timestamp      0
event_name           0
user_no              0
item_no              0
device_type          0
mobile_brand_name    0
country              0
region               0
platform             0
rating               0
birth_date           0
gender               0
age                  0
item_name            0
image_name           0
price                0
category1_code       0
category1_name       0
category2_code       0
category2_name       0
category3_code       0
category3_name       0
brand_no             0
brand_name           0
dtype: int64

In [13]:
product.head()

,item_no,item_name,image_name,price,category1_code,category1_name,category2_code,category2_name,category3_code,category3_name,brand_no,brand_name
0,++/211gtgspgB1TaPWEUQw==,베스트 코트\n( Vest Coat )_lavender,6842e3f320ef4459fac54dcea70c8245,1150000.0,243100100,의류,243101100,아우터,243101103,코트,4308.0,FOCEPS
1,++4eWFkX4oCtVYiKi34/Dw==,21SS 넥 페이스 패치 기모 맨투맨 블랙 CI0062 AHJ,1bdc06804580fddfbe46d4e2af140f08,466830.0,243100100,의류,243102100,상의,243102103,티셔츠,9302.0,Acne Studios_바이스트
2,++BoE+rYvLTJEAXMQFQLpQ==,[서울웰컴티 선물 세트] 일상 옥수수차,cb4a9bb103f9bd84d5d126ad4acec74b,15000.0,249100100,식품,249105100,음료,249105105,차류,4775.0,Urbanbookshop
3,++RIzj1vpFLWgoX7LgCfLQ==,PLAIN SQUARE BAG S (IV),4c3f75e886c8114d7d00d19a4f0c34d5,118000.0,244100100,잡화,244102100,가방,244102104,토트백,2546.0,zisoo
4,++W7/YMlPMwbAfyyJZEuIw==,클리어 바디미스트 150ml,5f136d4e25c837c3b658de16ac2156d9,27000.0,247100100,화장품/미용,247109100,바디케어,247109104,바디미스트,12986.0,밀크바오밥


In [ ]:
cat_product = product.copy()

cat_product['cat'] = cat_product['category1_name'] + " " + cat_product['category2_name']
cat_product['cat'] = cat_product['cat'] + " " + cat_product['category3_name']
cat_product

In [8]:
cat_product = cat_product.sample(frac=0.05, random_state=42)
cat_product = cat_product.drop_duplicates()
cat_product = cat_product.dropna()
cat_product['cat'] = cat_product['cat'].fillna('')

tfidf = TfidfVectorizer(stop_words='english') 
tfidf_matrix = tfidf.fit_transform(cat_product['cat'])

In [9]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim = pd.DataFrame(cosine_sim, index=cat_product.index, columns=cat_product.index)

In [10]:
indices = pd.Series(cat_product.index, index=cat_product['item_name'])
indices

item_name
Stream Dress_Sky                                                195722
UNISEX CUT-OFF DT LOGO SWEATSHORTS LIGHT MGREY_M_UDPA1E111G1      8322
당일발송)와이쓰리 GT5256 유틸리티 로고 스윔팬츠                                     3745
라탄 타원 미니 바구니 / 오발 트레이 정리함 인테리어 소품                               122497
버켄스탁 키즈 아리조나 일렉트릭 메탈릭실버 1019147                                 109624
                                                                 ...  
내열유리 머그잔1P 430ml (M-01)                                         184816
Modern Square Chain Bracelet                                    276474
Reversible linen blouse - black                                  49722
ABSL AIR-90 전용 카본필터                                              60902
플루크 마운틴 그래픽 실루엣 반팔티셔츠 FST151 / 4color M                          50215
Length: 14165, dtype: int64

In [11]:
def content_recommender(item_name, n_of_recomm):
  idx = indices[item_name]
  sim_scores = cosine_sim[idx].sort_values(ascending=False)[1:n_of_recomm]

  return cat_product.loc[sim_scores.index]['item_name']

print(content_recommender('Stream Dress_Sky', 5))
print('-' * 30)
print(content_recommender('Modern Square Chain Bracelet', 10))

201214    Ann Shirring Check Dress in Khaki
254995           V-NECK FLARE ONEPIECE_NAVY
386                         하와이안 레트로 원피스_핑크
92369               floral color P dress PK
Name: item_name, dtype: object
------------------------------
221835                        Basic Bangle_Beige [YA103_BG]
205319    [Silver925] TN051 Sleek curved line simple bra...
226605                                             실버 체인 팔찌
25231             Half And Blue Cord Bracelet Ib61 [Silver]
56473                                            핑퐁 블랙 원석팔찌
76433     Smile blue green string Bracelet 실버코인 블루 그린 실팔...
20081         [판도라 PANDORA] 587125CZ 모먼트 슬라이딩 스네이크 체인 로즈 팔찌
208145                             Curvilinear Archives 39A
172348                                          트로피컬 더블링 팔찌
Name: item_name, dtype: object
